In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage


In [2]:
load_dotenv()
import os
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    groq_api_key = groq_api_key,
    model_name = "openai/gpt-oss-120b",
)

In [3]:
#create a State

class LLMState(TypedDict):
    question: str
    answer: str


In [4]:
def qa_llm(state: LLMState) -> LLMState:
    
    question = state['question']

    prompt = f'Answer the following Question {question}'

    answer = llm.invoke([
        HumanMessage(content=prompt)
    ])

    state['answer'] = answer.content

    return state 

In [5]:
#create a graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node("qa_llm",qa_llm)

# add edges 
graph.add_edge(START,"qa_llm")
graph.add_edge("qa_llm",END)

# Compile
workflow = graph.compile()

In [6]:
# execute the graph
initial_state = {"question":"How far is the Moon from the Earth ?"}

final_state = workflow.invoke(initial_state)

print(final_state)


{'question': 'How far is the Moon from the Earth ?', 'answer': 'The Moon orbits the Earth at an average distance of **about\u202f384\u202f400\u202fkilometers (≈238\u202f900\u202fmiles)**.\n\n- **Perigee (closest point)**: roughly\u202f363\u202f300\u202fkm (≈225\u202f600\u202fmi)  \n- **Apogee (farthest point)**: roughly\u202f405\u202f500\u202fkm (≈252\u202f000\u202fmi)\n\nBecause the Moon’s orbit is slightly elliptical, the actual distance varies within this range as it circles the Earth roughly once every 27.3\u202fdays (sidereal period) or 29.5\u202fdays (synodic period, from new moon to new moon).'}
